# Implementing the `MiniBach` model - Part 4

In this step, we use the model trained on Part 3 to generate the three lower voices for a given soprano melodic line.

The model trained during Part 3 is

In [115]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import music21

In [116]:
model = tf.keras.models.load_model('trained_model.h5')

In [117]:
given_melody = '''
C4   --   --   --   |G4   --   --   --   |C4   --   D4   --   |C4   --   --   --   |
E4   --   --   --   |--   --   --   --   |F4   --   --   --   |--   --   --   --   |
G4   --   --   --   |--   --   --   --   |C4   --   --   --   |--   --   --   --   |
D4   --   --   --   |--   --   --   --   |--   --   --   --   |--   --   --   --   |
'''

In [118]:
given_melody = given_melody.replace('\n', '').replace('|', '')

In [119]:
tokens = given_melody.split()

In [120]:
SOPRANO_MIN = 57
SOPRANO_MAX = 81

ALTO_MIN = 52
ALTO_MAX = 74

TENOR_MIN = 48
TENOR_MAX = 69

BASS_MIN = 36
BASS_MAX = 64

ranges = {
    'soprano': {midinumber: (midinumber - SOPRANO_MIN + 1) for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {midinumber: (midinumber - ALTO_MIN + 1) for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {midinumber: (midinumber - TENOR_MIN + 1) for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {midinumber: (midinumber - BASS_MIN + 1) for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

reverse_ranges = {
    'soprano': {(midinumber - SOPRANO_MIN + 1): midinumber for midinumber in range(SOPRANO_MIN, SOPRANO_MAX + 1)},
    'alto': {(midinumber - ALTO_MIN + 1): midinumber for midinumber in range(ALTO_MIN, ALTO_MAX + 1)},
    'tenor': {(midinumber - TENOR_MIN + 1): midinumber for midinumber in range(TENOR_MIN, TENOR_MAX + 1)},
    'bass': {(midinumber - BASS_MIN + 1): midinumber for midinumber in range(BASS_MIN, BASS_MAX + 1)},
}

In [121]:
def encode_note(n, rang):
    if n == '--' or n == 'Rest':
        ret = 0
    else:
        note = music21.note.Note(n)
        ret = ranges[rang][note.pitch.midi]
    return ret

def one_hot_encode(idx, rang):
    length = len(ranges[rang].values())
    ret = [0] * (length + 1)
    ret[idx] = 1
    return ret

In [123]:
s = [encode_note(n, 'soprano') for n in tokens] 
x = np.array([[one_hot_encode(idx, 'soprano') for idx in s]])    
x = x.reshape(1, -1)

In [124]:
predictions = model.predict(x)

predictions = predictions.reshape(-1)

soprano = x.reshape(64, -1)
alto = predictions[:1536].reshape(64, -1)
tenor = predictions[1536:3008].reshape(64, -1)
bass = predictions[3008:4928].reshape(64, -1)

In [125]:
def decode_note(n, rang):
    if n == 0:
        ret = '--'
    else:
        note = music21.note.Note(type='16th')        
        note.pitch.midi = reverse_ranges[rang][n]        
        ret = note
    return ret

In [126]:
generation = {
    'soprano': [],
    'alto': [],
    'tenor': [],
    'bass': []
}

for sixteenth in range(64):
    soprano_note = decode_note(np.argmax(soprano[sixteenth]), 'soprano')
    if soprano_note == '--':
        last_note = generation['soprano'][-1]
        soprano_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
        if last_note.tie:
            soprano_note.tie = music21.tie.Tie('continue')
        else:
            last_note.tie = music21.tie.Tie('start')
            generation['soprano'][-1] = last_note
            soprano_note.tie = music21.tie.Tie('continue')
    else:
        if sixteenth > 0:
            last_note = generation['soprano'][-1]
            if last_note.tie:
                last_note.tie.type = 'end'
    generation['soprano'].append(soprano_note)
    
    alto_note = decode_note(np.argmax(alto[sixteenth]), 'alto')
    if alto_note == '--':
        last_note = generation['alto'][-1]
        alto_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
        if last_note.tie:
            alto_note.tie = music21.tie.Tie('continue')
        else:
            last_note.tie = music21.tie.Tie('start')
            generation['alto'][-1] = last_note
            alto_note.tie = music21.tie.Tie('continue')
    else:
        if sixteenth > 0:
            last_note = generation['alto'][-1]
            if last_note.tie:
                last_note.tie.type = 'end'
    generation['alto'].append(alto_note)
    
    tenor_note = decode_note(np.argmax(tenor[sixteenth]), 'tenor')
    if tenor_note == '--':
        last_note = generation['tenor'][-1]
        tenor_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
        if last_note.tie:
            tenor_note.tie = music21.tie.Tie('continue')
        else:
            last_note.tie = music21.tie.Tie('start')
            generation['tenor'][-1] = last_note
            tenor_note.tie = music21.tie.Tie('continue')
    else:
        if sixteenth > 0:
            last_note = generation['alto'][-1]
            if last_note.tie:
                last_note.tie.type = 'end'
    generation['tenor'].append(tenor_note)
    
    bass_note = decode_note(np.argmax(bass[sixteenth]), 'bass')
    if bass_note == '--':
        last_note = generation['bass'][-1]
        bass_note = music21.note.Note(last_note.pitch.nameWithOctave, type='16th')
        if last_note.tie:
            bass_note.tie = music21.tie.Tie('continue')
        else:
            last_note.tie = music21.tie.Tie('start')
            generation['bass'][-1] = last_note
            bass_note.tie = music21.tie.Tie('continue')
    else:
        if sixteenth > 0:
            last_note = generation['bass'][-1]
            if last_note.tie:
                last_note.tie.type = 'end'
    generation['bass'].append(bass_note)        

In [127]:
df = pd.DataFrame(generation)

In [128]:
df

,soprano,alto,tenor,bass
0,<music21.note.Note C>,<music21.note.Note E>,<music21.note.Note A>,<music21.note.Note C>
1,<music21.note.Note C>,<music21.note.Note E>,<music21.note.Note A>,<music21.note.Note C>
2,<music21.note.Note C>,<music21.note.Note E>,<music21.note.Note A>,<music21.note.Note C>
3,<music21.note.Note C>,<music21.note.Note E>,<music21.note.Note A>,<music21.note.Note C>
4,<music21.note.Note G>,<music21.note.Note G>,<music21.note.Note C>,<music21.note.Note D>
...,...,...,...,...
59,<music21.note.Note D>,<music21.note.Note D>,<music21.note.Note A>,<music21.note.Note D>
60,<music21.note.Note D>,<music21.note.Note D>,<music21.note.Note A>,<music21.note.Note D>
61,<music21.note.Note D>,<music21.note.Note D>,<music21.note.Note A>,<music21.note.Note D>
62,<music21.note.Note D>,<music21.note.Note D>,<music21.note.Note A>,<music21.note.Note D>


In [129]:
s = music21.stream.Stream()
s.append(df.soprano.to_list())
a = music21.stream.Stream()
a.append(df.alto.to_list())
t = music21.stream.Stream()
t.append(df.tenor.to_list())
b = music21.stream.Stream()
b.append(df.bass.to_list())
stream = music21.stream.Stream([s,a,t,b])

In [130]:
stream.write('musicxml', 'here.musicxml')

'/home/napulen/dev/MiniBach/here.musicxml'